In [3]:
#### from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pickle
import time
options = Options()
options.add_argument("--start-maximized")
options.add_argument("window-size=1920x1080")
options.add_argument("lang=ko_KR")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.instagram.com/accounts/login/")
print("수동으로 인증 시작!")
time.sleep(30)
with open("instagram_cookies.pkl", "wb") as f :
    pickle.dump(driver.get_cookies(), f)
print("쿠키 저장 완료")
driver.quit()
# cookie, wb 형태로 저장

수동으로 인증 시작!
쿠키 저장 완료


In [7]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pickle
import time
# [ADD] 최소 변경: 500개+ 수집 & CSV 저장용
import random, re, csv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# [FIX] StaleElementReferenceException 임포트
from selenium.common.exceptions import StaleElementReferenceException

# ================= 추가 설정값 =================
TARGET_CAPTIONS = 10               # 최소 수집 목표(500개 이상) - 테스트용으로 10으로 유지
MAX_SCROLLS = 10                   # 과도 스크롤 방지용 상한
SCROLL_WAIT = (2.0, 3.5)           # 스크롤 후 대기(랜덤)
VISIT_WAIT = (2.0, 3.5)            # 게시물 방문 후 대기(랜덤)
CSV_PATH = "instagram_captions.csv"  # 저장 파일명(utf-8-sig)
# =================================================
options = Options()
options.add_argument("--start-maximized")
options.add_argument("window-size=1920x1080")
options.add_argument("lang=ko_KR")
# headless 옵션 추가 (필요시 주석 해제)
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36")


service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# driver 생성 직후
wait = WebDriverWait(driver, 10)

driver.get("https://www.instagram.com/")

try:
    with open("instagram_cookies.pkl", "rb") as f :
        cookies = pickle.load(f)

    for cookie in cookies :
        # [ADD] expiry가 float이면 Selenium이 거부하는 경우가 있어 int로 보정(문제 없으면 그대로 동작)
        if "expiry" in cookie and isinstance(cookie["expiry"], float):
            cookie["expiry"] = int(cookie["expiry"])
        try:
            driver.add_cookie(cookie)
        except Exception:
            pass
    
    driver.refresh()
    print("쿠키 로드 및 로그인 시도...")
    time.sleep(5)

except FileNotFoundError:
    print("instagram_cookies.pkl 파일을 찾을 수 없습니다. 로그인이 필요할 수 있습니다.")
    # 쿠키 파일이 없으면 수동 로그인을 위해 60초 대기 (예시)
    # print("60초 내에 수동으로 로그인해주세요...")
    # time.sleep(60)
    # print("수동 로그인 시간 종료. 크롤링을 계속합니다.")
    pass


# import re # (이미 위에서 import 됨)
def extract_caption_and_hashtags():
    """
    - 캡션: article 내부의 <h1 dir="auto"> 우선 (요청사항 반영)
    - 보조: 첫 li/span, 마지막 fallback 으로 img[alt]
    - 해시태그: article 내부의 a[href^='/explore/tags/'] 텍스트 (요청사항 반영)
    """
    # 페이지 로드 보장
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "article")))
    except:
        print("게시물 <article> 요소를 찾는 데 실패했습니다.")
        pass # 실패해도 일단 진행

    # '더 보기' 펼치기(있으면)
    for t in ["더 보기", "더보기", "More", "more", "… 더보기", "… more"]:
        try:
            # XPath를 더 유연하게 수정 (span의 text() 또는 button의 text() 또는 div(@role='button')가 일치하는 경우)
            btn = driver.find_element(By.XPATH, f"//span[normalize-space()='{t}']/.. | //button[normalize-space()='{t}'] | //div[normalize-space()='{t}' and @role='button']")
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.3)
            break # 하나라도 성공하면 중지
        except:
            pass
            
    caption = ""
    # 1) 현재 UI: <h1 dir="auto"> 안에 본문
    #    (제공된 HTML 조각과 일치하는 가장 우선순위 높은 선택자)
    try:
        el = driver.find_element(By.CSS_SELECTOR, "article h1[dir='auto']")
        caption = el.text.strip()
    except:
        pass

    # 2) 예전 구조: 첫 댓글/본문 span
    if not caption:
        try:
            # XPath는 인스타그램 UI 변경에 취약할 수 있으나, 기존 로직 유지
            el = driver.find_element(By.XPATH, "//article//ul//li[1]//div//div//span")
            caption = el.text.strip()
        except:
            pass

    # 3) 마지막 fallback: 이미지 alt
    if not caption:
        try:
            # article 내부의 메인 이미지 alt 값으로 한정 시도
            img = driver.find_element(By.XPATH, "//article//div[@role='presentation']//img[@alt]")
            caption = (img.get_attribute("alt") or "").strip()
        except:
            # 위가 실패하면 기존 로직(페이지의 첫 alt)
            try:
                img = driver.find_element(By.XPATH, "//img[@alt]")
                caption = (img.get_attribute("alt") or "").strip()
            except:
                pass

    # 해시태그는 DOM에서 직접 수집(가장 확실)
    # (제공된 HTML 조각과 일치하는 방식)
    hashtags = []
    try:
        # article 내부로 한정하여 더 정확하게 검색
        tag_els = driver.find_elements(By.XPATH, "//article//a[starts-with(@href, '/explore/tags/')]")
        for a in tag_els:
            t = (a.text or "").strip()
            if t.startswith("#"):
                t = t[1:] # '#' 제거
            if t and t not in hashtags:
                hashtags.append(t)
    except:
        pass

    # 그래도 없으면 캡션에서 정규식으로 보조 추출
    if not hashtags and caption:
        # 정규식 수정: #뒤에 바로 붙는 한글/영문/숫자/_
        found_tags = re.findall(r"#([0-9A-Za-z_가-힣]+)", caption)
        for t in found_tags:
            if t not in hashtags:
                hashtags.append(t)
                
    return caption, hashtags

# 팝업 처리
try :
    # "나중에 하기" 버튼이 로드될 때까지 잠시 대기
    not_now_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()='나중에 하기'] | //div[@role='button' and text()='나중에 하기']")))
    not_now_button.click()
    print("'나중에 하기' 팝업 1 클릭됨.")
    time.sleep(2)
except :
    print("알림 설정 팝업(1)이 없거나 클릭에 실패했습니다.")
    pass

try :
    # "나중에 하기" 알림 설정 팝업이 하나 더 있을 수 있음 (짧은 대기 시간)
    WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//button[text()='나중에 하기'] | //div[@role='button' and text()='나중에 하기']"))).click()
    print("'나중에 하기' 팝업 2 클릭됨.")
    time.sleep(2)
except :
    print("두 번째 알림 설정 팝업(2)이 없거나 클릭에 실패했습니다.")
    pass


keyword = "도시락"
# (요청대로 기존 흐름 유지: 태그 URL 구성 방식 변경 없음)
driver.get(f"https://www.instagram.com/explore/tags/{keyword}")
print(f"'{keyword}' 태그 페이지 로드 중...")

# 페이지 로드 대기
try:
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "a")))
except Exception as e:
    print(f"태그 페이지 로드 실패: {e}")
    driver.quit()
    # exit() # 스크립트 종료

# ================== [REPLACE] 링크 수집부(3회→대량 수집) ==================
post_links = []
seen = set()
scrolls = 0
# TARGET_CAPTIONS의 1.2배 또는 최대 500개 정도의 버퍼를 두는 것이 좋습니다.
# 여기서는 기존 로직 1.2배를 유지합니다.
target_link_count = int(TARGET_CAPTIONS * 1.2)
if target_link_count < 50: # 최소한 50개는 수집 시도
    target_link_count = 50

print(f"게시물 링크 수집 시작 (목표: {target_link_count}개, 최대 스크롤: {MAX_SCROLLS}회)")

while len(post_links) < target_link_count and scrolls < MAX_SCROLLS:
    links = driver.find_elements(By.TAG_NAME, "a")
    new_links_found = 0
    
    # [FIX] StaleElementReferenceException 처리를 위해 try...except 블록 추가
    for link in links:
        try:
            href = link.get_attribute("href")
            # /p/(게시물) 또는 /reel/(릴스) 링크만 수집
            if href and ("/p/" in href or "/reel/" in href):
                if href not in seen:
                    seen.add(href)
                    post_links.append(href)
                    new_links_found += 1
        except StaleElementReferenceException:
            # 요소를 찾는 것과 속성을 가져오는 것 사이에 DOM이 변경되었습니다.
            # 이 요소는 무시하고 계속 진행합니다.
            pass
    
    if new_links_found == 0 and scrolls > 3: # 3번 스크롤에도 새 링크가 없으면
        print("새로운 링크를 찾지 못해 스크롤을 중단합니다.")
        break # 몇 번 스크롤해도 새 링크가 없으면 중단

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(*SCROLL_WAIT))
    scrolls += 1
    
    if scrolls % 5 == 0:
        print(f"[스크롤 {scrolls}/{MAX_SCROLLS}] 링크 수집: {len(post_links)}개 / 목표: {target_link_count}")
# ========================================================================

results = []
print(f"\n총 {len(post_links)}개의 링크에서 캡션 수집을 시작합니다 (목표: {TARGET_CAPTIONS}개).")

for i, url in enumerate(post_links):
    if len(results) >= TARGET_CAPTIONS:
        print(f"목표 캡션 {TARGET_CAPTIONS}개를 모두 수집하여 중단합니다.")
        break
    
    print(f"[{len(results) + 1}/{TARGET_CAPTIONS}] 방문 중: {url}")
    try:
        driver.get(url)
        # 페이지 로드를 위해 명시적 대기 (article이 나타날 때까지)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "article")))
        # 추가로 VIS_WAIT의 일부 시간 대기 (랜덤성 부여)
        time.sleep(random.uniform(VISIT_WAIT[0] * 0.5, VISIT_WAIT[1] * 0.5)) 
        
        caption, hashtags = extract_caption_and_hashtags()
        
        if not caption and not hashtags:
            print(f"  -> 캡션/해시태그 없음 (비공개 또는 삭제된 게시물 추정)")
            continue

        results.append({"url": url, "caption": caption, "hashtags": hashtags})

    except Exception as e:
        print(f"  -> 캡션/해시태그 추출 중 오류 발생 : {url} | {e}")
        continue

driver.quit()

# 기존 프린트 유지 (결과가 너무 많으면 일부만 출력하도록 수정)
print("\n" + "="*20 + " 수집 결과 (최대 10건) " + "="*20)
for r in results[:10]: # 최대 10개만 출력
    print(f"URL: {r['url']}")
    print(f"Caption: {r['caption'][:150]}...") # 캡션이 길면 150자만
    print(f"Hashtags: {';'.join(r['hashtags'])}")
    print("="*60)

# ================== [FIX] CSV 저장(utf-8-sig) ==================
# 'parse_hashtags' 함수는 extract_caption_and_hashtags에서 이미
# 더 정확한 방식(DOM 검색)으로 처리하므로 여기서는 불필요합니다.
# def parse_hashtags(text): ... (삭제)

print(f"\nCSV 파일 저장 중... ({CSV_PATH})")
with open(CSV_PATH, "w", newline="", encoding="utf-8-sig") as f:
    w = csv.writer(f)
    w.writerow(["url", "caption", "hashtags"])  # 헤더
    
    for r in results:
        # [수정] extract_caption_and_hashtags에서 반환된 'hashtags' 리스트(r["hashtags"])를
        # 직접 사용하여 세미콜론(;)으로 join합니다.
        w.writerow([r["url"], r["caption"], ";".join(r["hashtags"])])

print(f"[CSV 저장 완료] {CSV_PATH} (총 {len(results)}건)")
# ================================================================




쿠키 로드 및 로그인 시도...
알림 설정 팝업(1)이 없거나 클릭에 실패했습니다.
두 번째 알림 설정 팝업(2)이 없거나 클릭에 실패했습니다.
'도시락' 태그 페이지 로드 중...
게시물 링크 수집 시작 (목표: 50개, 최대 스크롤: 10회)

총 63개의 링크에서 캡션 수집을 시작합니다 (목표: 10개).
[1/10] 방문 중: https://www.instagram.com/p/DIxJF_kz7ja/
  -> 캡션/해시태그 추출 중 오류 발생 : https://www.instagram.com/p/DIxJF_kz7ja/ | Message: 
Stacktrace:
	GetHandleVerifier [0x0xebfe43+66515]
	GetHandleVerifier [0x0xebfe84+66580]
	(No symbol) [0x0xcadc48]
	(No symbol) [0x0xcf8704]
	(No symbol) [0x0xcf8aab]
	(No symbol) [0x0xd3f482]
	(No symbol) [0x0xd1b214]
	(No symbol) [0x0xd3cba7]
	(No symbol) [0x0xd1afc6]
	(No symbol) [0x0xcec2ca]
	(No symbol) [0x0xced154]
	GetHandleVerifier [0x0x1117353+2521315]
	GetHandleVerifier [0x0x11122d3+2500707]
	GetHandleVerifier [0x0xee7c94+229924]
	GetHandleVerifier [0x0xed81f8+165768]
	GetHandleVerifier [0x0xedecad+193085]
	GetHandleVerifier [0x0xec8158+100072]
	GetHandleVerifier [0x0xec82f0+100480]
	GetHandleVerifier [0x0xeb25aa+11066]
	BaseThreadInitThunk [0x0x767c5d49+25]
	RtlInitializeEx

KeyboardInterrupt: 

In [9]:
options = Options()
options.add_argument("--start-maximized")
options.add_argument("window-size=1920x1080")
options.add_argument("lang=ko_KR")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# [FIX] driver 생성 후로 이동
wait = WebDriverWait(driver, 10)

# [ADD] URL 인코딩
from urllib.parse import quote

keyword = "도시락"
driver.get(f"https://www.instagram.com/explore/tags/{quote(keyword)}")

#3) 캡션/해시태그 추출 함수 보강 (h1 + h1 내부 a[href^='/explore/tags/'])
def extract_caption_and_hashtags():
    """
    - 캡션: article 내부 <h1 dir="auto"> 우선
    - 해시태그: 1) h1 내부 a[href^='/explore/tags/'] 2) article 전역 해시태그 a 3) 마지막으로 정규식
    """
    # 페이지 로드 보장
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "article")))
    except:
        pass

    # '더 보기' 펼치기(문구 변화 대응)
    for t in ["더 보기", "더보기", "More", "more", "… 더보기", "… more"]:
        try:
            btn = driver.find_element(By.XPATH, f"//button[.//span[normalize-space()='{t}'] or normalize-space()='{t}']")
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.3)
            break
        except:
            pass

    caption = ""
    hashtags = []

    # 1) 현재 UI: <h1 dir="auto"> 본문
    try:
        el = driver.find_element(By.CSS_SELECTOR, "article h1[dir='auto']")
        caption = el.text.strip()

        # [FIX] 우선순위: h1 내부 해시태그 a 태그
        try:
            h1_tags = el.find_elements(By.CSS_SELECTOR, "a[href^='/explore/tags/']")
            for a in h1_tags:
                t = (a.text or "").strip()
                if t.startswith("#"):
                    t = t[1:]
                if t and t not in hashtags:
                    hashtags.append(t)
        except:
            pass
    except:
        pass

    # 2) 예전 구조: 첫 li/span (캡션만 보조)
    if not caption:
        try:
            el = driver.find_element(By.XPATH, "//article//ul//li[1]//div//div//span")
            caption = el.text.strip()
        except:
            pass

    # 3) article 전역 해시태그 a 태그 (h1에서 못 찾았을 때만)
    if not hashtags:
        try:
            tag_els = driver.find_elements(By.XPATH, "//article//a[starts-with(@href, '/explore/tags/')]")
            for a in tag_els:
                t = (a.text or "").strip()
                if t.startswith("#"):
                    t = t[1:]
                if t and t not in hashtags:
                    hashtags.append(t)
        except:
            pass

    # 4) 마지막 fallback: 이미지 alt로 캡션 보조
    if not caption:
        try:
            img = driver.find_element(By.XPATH, "//img[@alt]")
            caption = (img.get_attribute("alt") or "").strip()
        except:
            pass

    # 5) 그래도 해시태그가 비어있으면 정규식 보조
    if not hashtags and caption:
        rx = re.findall(r"#([0-9A-Za-z_가-힣]+)", caption)
        for t in rx:
            if t and t not in hashtags:
                hashtags.append(t)

    return caption, hashtags

# [KEEP] 기존 parse_hashtags는 보조용으로 남겨둠
def parse_hashtags(text):
    tags = re.findall(r"#([0-9A-Za-z_가-힣_]+)", text or "")
    seen = set(); uniq = []
    for t in tags:
        if t not in seen:
            seen.add(t); uniq.append(t)
    return uniq

with open(CSV_PATH, "w", newline="", encoding="utf-8-sig") as f:
    w = csv.writer(f)
    w.writerow(["url", "caption", "hashtags"])  # hashtags는 ;로 join
    for r in results:
        # [FIX] DOM에서 수집한 r["hashtags"] 우선 사용, 비었으면 캡션에서 보조 추출
        tags = r.get("hashtags") or parse_hashtags(r.get("caption", ""))
        w.writerow([r["url"], r["caption"], ";".join(tags)])

PermissionError: [Errno 13] Permission denied: 'instagram_captions.csv'